# Base Video Pipeline

Load overlay

In [13]:
from pynq import Overlay
from pynq.lib.video import *
from pynq.lib import *

base = Overlay('km_inp8b_outp32b_gray2rgba.bit', ignore_version=True)
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out


Configure HDMI

In [14]:
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
print(hdmi_in.colorspace)


# monitor configuration: 1280*720 @ 60Hz ;32 voor raspbpi configged at 720p 50hz
Mode = VideoMode(1280,720,8)
print(hdmi_in.mode)
print(hdmi_out.mode)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_GRAY)
hdmi_out.start()

hdmi_in.tie(hdmi_out)
print(hdmi_out.colorspace)



[0.11328125, 0.5859375, 0.296875, 0.5, -0.328125, -0.16796875, -0.078125, -0.41796875, 0.5, -0.0, 0.5, 0.5]
VideoMode: width=1280 height=720 bpp=8
VideoMode: width=1280 height=720 bpp=8
[1.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [ ]:
Gepruts met colorspace conversion. Niet uitvoeren voor normaal canny edging

In [32]:
colorspace_in = base.video.hdmi_in.color_convert
colorspace_out = base.video.hdmi_out.color_convert

bgr2rgb = [-2, 1, 1,
           1, 1, 1, 
           1, 1, -2,
           0, 0, 0]

brg2gray = [0.11328125, 0.5859375, 0.296875, 0.5, -0.328125, -0.16796875, -0.078125, -0.41796875, 0.5, -0.0, 0.5, 0.5]

colorspace_in.colorspace = brg2gray
colorspace_out.colorspace = bgr2rgb

colorspace_in.colorspace

print(hdmi_out.colorspace)


[-2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -2.0, -0.0, -0.0, -0.0]


Gepruts met pixelpack conversion. Niet uitvoeren voor normaal canny edging

In [4]:
pixel_in = base.video.hdmi_in.pixel_pack
pixel_out = base.video.hdmi_out.pixel_unpack

pixel_in.bits_per_pixel = 8
pixel_out.bits_per_pixel = 8

pixel_in.bits_per_pixel

8


Write configuration data (for the `hfilt` function)

In [ ]:
from pynq.mmio import MMIO
msk = MMIO(0x43C7_0000, 0x1000)
msk.write(0x10, 0x00)
msk.write(0x18, 0xFF)
msk.write(0x20, 0x00)
#for y in range(0,720):
    #msk.write(0x10, y)
#    for x in range(0,1280):
#        msk.write(0x18, x)
#msk.write(0x20, 897987)

Don't forget to close the HDMI connection!

In [ ]:
hdmi_in.tie(hdmi_out)

In [ ]:
hdmi_out.close()
hdmi_in.close()
